# GNPy Simulation

In [ ]:
import os
from pathlib import Path
from gnpy.core import utils, info, elements, network, equipment
from gnpy.topology import request
from gnpy.tools.json_io import load_network, load_equipment

In [ ]:
import networkx as nx
from pyvis import network as pvn

def plot_graph(network):
    """
    Plots a NetworkX DiGraph using PyVis and saves the HTML file.

    Args:
        network (networkx.DiGraph): The input NetworkX DiGraph.

    Returns:
        None
    """
    # Convert node IDs to strings
    G = nx.relabel_nodes(network, {n: str(n) for n in network.nodes()})

    # Set the weight attribute to 1 for all edges
    for u, v in G.edges():
        G.edges[u, v]['weight'] = 0

    # Create a PyVis network object from the graph
    net = pvn.Network(notebook=True, directed=True)

    # Define custom node and edge attribute dictionaries
    node_attrs = {}  # No node attributes
    edge_attrs = {}  # No edge attributes

    net.from_nx(G)

    # Show some additional options
    net.show_buttons(filter_=['physics'])
    net.show('network_plot.html')

In [ ]:
ROOT_PATH = Path(os.getcwd())
RESOURCES_PATH = ROOT_PATH
topo = RESOURCES_PATH / "tnc24_topo.json"
conf = RESOURCES_PATH / "tnc24_eqpt.json"
lab_eqpt = load_equipment(conf)
lab_net = load_network(topo, lab_eqpt)
plot_graph(lab_net)

In [ ]:
si = info.create_input_spectral_information(f_min = lab_eqpt['SI']['default'].f_min, 
                                            f_max = lab_eqpt['SI']['default'].f_max, 
                                            roll_off = lab_eqpt['SI']['default'].roll_off, 
                                            baud_rate = lab_eqpt['SI']['default'].baud_rate, 
                                            power = utils.dbm2watt(lab_eqpt['SI']['default'].power_dbm), 
                                            spacing = lab_eqpt['SI']['default'].spacing, 
                                            tx_osnr = lab_eqpt['SI']['default'].tx_osnr)

In [ ]:
roadms = [r for r in lab_net.nodes() if isinstance(r, elements.Roadm)]
transceivers = [t for t in lab_net.nodes() if isinstance(t, elements.Transceiver)]
total_power = lab_eqpt['SI']['default'].power_dbm + utils.lin2db(si.number_of_channels)

In [ ]:
for roadm in roadms:
    network.set_roadm_ref_carrier(roadm, lab_eqpt) #this saves the baud_rate and the spacing defined in equipment['SI'] in each "roadm"
    network.set_roadm_per_degree_targets(roadm, lab_net) #set the 'per_degree_pch_psd'...don't know why it is not done before
for roadm in roadms + transceivers:
    network.set_egress_amplifier(lab_net, roadm, lab_eqpt, pref_ch_db=lab_eqpt['SI']['default'].power_dbm, 
                                 pref_total_db=total_power, verbose=True)
for roadm in roadms:
    network.set_roadm_input_powers(lab_net, roadm, lab_eqpt, lab_eqpt['SI']['default'].power_dbm)
for fiber in [f for f in lab_net.nodes() if isinstance(f, (elements.Fiber, elements.RamanFiber))]:
    network.set_fiber_input_power(lab_net, fiber, lab_eqpt, lab_eqpt['SI']['default'].power_dbm)

In [ ]:
amps = [r for r in lab_net.nodes() if isinstance(r, elements.Edfa)]

In [ ]:
source_node = "TX"
dest_node = "RX"
source = next((node for node in lab_net.nodes() if node.uid == source_node and isinstance(node, elements.Transceiver)), None)
dest = next((node for node in lab_net.nodes() if node.uid == dest_node and isinstance(node, elements.Transceiver)), None)

params = {}
params['request_id'] = 0
params['source'] = source.uid
params['destination'] = dest.uid
params['nodes_list'] = [dest.uid]
params['loose_list'] = ['STRICT']

class MyParams:
    def __init__(self, **entries):
        self.__dict__.update(entries)

params = MyParams(**params)

In [ ]:
path = request.compute_constrained_path(lab_net, params)
path

In [ ]:
for i, el in enumerate(path):
    if isinstance(el, elements.Roadm):
        si = el(si, degree=path[i + 1].uid, from_degree=path[i - 1].uid)
    else:
        si = el(si)    
    print(el.uid, vars(si))

In [ ]:
utils.mean(dest.snr)

In [ ]:
utils.mean(dest.snr_01nm)

In [ ]:
dest.snr[48]

# Simulation vs Measurement

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
from gnpy.core import utils
from scipy.optimize import curve_fit, fsolve
from scipy.special import ndtr, erfc

In [ ]:
ROOT_PATH = Path(os.getcwd())
BAUDRATE = 32 #GBaud
NOISE_BW =  12.5 #GHz
ROLLOFF = 1.15 #root raised cosine roll-off factor

In [ ]:
def b2b(x, a, b, c):
    gsnr = x
    snr_trx = c
    snr_tot = (gsnr * snr_trx) / (gsnr + snr_trx)
    return a * ndtr(- b * utils.sqrt(snr_tot))

In [ ]:
df = pd.read_excel(ROOT_PATH / 'BER_vs_OSNR.xlsx', sheet_name='Sheet1', header=3)
osnr_01_db = df.values[:, 0]
ber = df.values[:, 1]
osnr = utils.db2lin(osnr_01_db) * NOISE_BW / BAUDRATE
osnr_db = utils.lin2db(osnr)
ber_db = utils.lin2db(ber)

In [ ]:
popt, pcov = curve_fit(b2b, xdata=osnr, ydata=ber, bounds=([0., 0., 10], [1., 1.2, 900]))
print(popt)
print(pcov)

In [ ]:
fit = go.Scatter(x=osnr_db, y=utils.lin2db(b2b(osnr, *popt)), mode='lines', name='fit')
measure = go.Scatter(x=osnr_db, y=ber_db, mode='markers', marker=dict(symbol='x',size=8), name='measures')
layout = go.Layout(title='', xaxis_title='OSNR', yaxis_title='pre-FEC BER')
fig = go.Figure(data=[fit, measure], layout=layout)
fig.show()

In [ ]:
MEASURED_BER = 0.03

In [ ]:
def func(x):
    return b2b(x, *popt) - MEASURED_BER
gsnr = fsolve(func, x0=20)

In [ ]:
gsnr_error = dest.snr[48] - utils.lin2db(gsnr[0])
ber_error = MEASURED_BER - b2b(utils.db2lin(dest.snr[48]), *popt)
print(f"GSNR estimation is {gsnr_error:.3f} dB higher than measurement.")
print(f"Pre-FEC BER estimation is {ber_error:.5f} lower than measurement.")